## Preparation

In [1]:
import bigframes
import bigframes.pandas as bpd

Enable the semantic operator experiment

In [2]:
bigframes.options.experiments.semantic_operators = True

/usr/local/google/home/chelsealin/src/bigframes3/bigframes/_config/experiment_options.py:33: UserWarning: Semantic operators are still under experiments, and are subject to change in the future.
  warnings.warn(


Prepare the LLM model. Here we are going to use Gemini 1.5 Flash.

In [3]:
import bigframes.ml.llm as llm
gemini_model = llm.GeminiTextGenerator(model_name=llm._GEMINI_1P5_FLASH_001_ENDPOINT)

/usr/local/google/home/chelsealin/src/bigframes3/bigframes/pandas/__init__.py:559: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return global_session.get_global_session()


## Semantic Filtering

In [4]:
df = bpd.DataFrame({'country': ['USA', 'Germany'], 'city': ['Seattle', 'Berlin']})
df.semantics.filter("{city} is the capital of {country}", gemini_model)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,country,city
1,Germany,Berlin


## Semantic Mapping

In [5]:
df = bpd.DataFrame(
        data={"ingredient_1": ["Burger Bun", "Soy Bean"], "ingredient_2": ["Beef Patty", "Bittern"]}
    )

In [6]:
df.semantics.map("What is the food made from {ingredient_1} and {ingredient_2}? One word only.", output_column="food", model=gemini_model)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,ingredient_1,ingredient_2,food
0,Burger Bun,Beef Patty,Burger
1,Soy Bean,Bittern,Tofu


## Semantic Joining

In [7]:
cities = bpd.DataFrame({'city': ['Seattle', 'Ottawa', 'Berlin', 'Shanghai', 'New Delhi']})
continents = bpd.DataFrame({'continent': ['North America', 'Africa', 'Asia']})

In [8]:
cities.semantics.join(continents, "{city} is in {continent}", gemini_model)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,city,continent
0,Seattle,North America
1,Ottawa,North America
2,Shanghai,Asia
3,New Delhi,Asia


### Self Joins

In [9]:
animals = bpd.DataFrame({'animal': ['cow', 'cat', 'spider', 'elephant']})

In [10]:
animals.semantics.join(animals, "{animal_left} generally weighs heavier than {animal_right}", gemini_model)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,animal_left,animal_right
0,cow,cat
1,cow,spider
2,cat,spider
3,elephant,cow
4,elephant,cat
5,elephant,spider


## Semantic Search

In [11]:
text_embedding_model = llm.TextEmbeddingGenerator(model_name="text-embedding-004")

In [12]:
df = bpd.DataFrame({"creatures": ["salmon", "sea urchin", "baboons", "frog", "chimpanzee"]})
df

,creatures
0,salmon
1,sea urchin
2,baboons
3,frog
4,chimpanzee


In [13]:
df.semantics.search("creatures", "monkey", top_k = 2, model = text_embedding_model, score_column='similarity score')

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,creatures,similarity score
2,baboons,0.773411
4,chimpanzee,0.781101


## Semantic Aggregation

In [4]:
df = bpd.DataFrame({
    "Movies": [
        "Titanic",
        "The Wolf of Wall Street",
        "Killers of the Flower Moon",
        "The Revenant",
        "Inception",
        "Shuttle Island",
        "The Great Gatsby",
    ],
    "Year": [1997, 2013, 2023, 2015, 2010, 2010, 2013],
})
df

,Movies,Year
0,Titanic,1997
1,The Wolf of Wall Street,2013
2,Killers of the Flower Moon,2023
3,The Revenant,2015
4,Inception,2010
5,Shuttle Island,2010
6,The Great Gatsby,2013


In [5]:
agg_df = df.semantics.agg("Find the shared first name of actors in {Movies}. One word answer.", model=gemini_model)
agg_df

/usr/local/google/home/chelsealin/src/bigframes3/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


0    Leonardo 

Name: Movies, dtype: string